In [2]:
import wordninja
import re
import csv
import pytesseract
from tempfile import TemporaryDirectory
from PIL import Image
from pathlib import Path 
from pdf2image import convert_from_path


pdf = Path()/"bini_dict.pdf" 


In [ ]:
def read_pdf2img(pdf_path):
    """
    Converts the PDF to images and writes the OCR output to a single text file.
    """
    output_folder = Path("output_folder")
    output_folder.mkdir(exist_ok=True)

    with TemporaryDirectory() as tmp:
        storage = Path(tmp)
        convert_from_path(
            pdf_path=Path(pdf_path),
            output_folder=storage,
            fmt='png',
            single_file=False,
            first_page=19,
            thread_count=4,
        )

        with open(output_folder/"output_file.txt", "w") as f:
            for x in sorted(storage.glob("*.png")): 
                text = pytesseract.image_to_string(Image.open(x))
                f.write(text + "\n\n")  # Add newlines between pages

read_pdf2img(pdf)


In [ ]:
# def clean_text():
#     """
#     extract bini word and definitions, clean both separately and store the output as csv
#     """
#     text = Path()/"output_folder/output_file.txt"

# NOT TESTED, I DO NOT KNOW WHAT THIS CODE DOES AT ALL, but by all means look into it - kinged

INPUT_TXT  = "Bini_mini.txt"   # path to your text file
OUTPUT_CSV = "bini_pairs.csv"  # output csv filename

def normalize_block(block: str) -> str:
    """Fix hyphenated line-breaks and join lines with spaces."""
    lines = [ln.strip() for ln in block.splitlines()]
    combined = []
    for ln in lines:
        if not combined:
            combined.append(ln)
        else:
            if combined[-1].endswith("-"):
                combined[-1] = combined[-1][:-1] + ln  # remove trailing '-' and glue
            else:
                combined.append(ln)
    return " ".join(combined)

def extract_pairs(text: str):
    # 1) Split the file into blocks separated by blank lines
    blocks = [b.strip() for b in re.split(r"\n\s*\n", text) if b.strip()]

    rows = []
    for block in blocks:
        nb = normalize_block(block)
        # Skip headers
        if nb.upper().startswith("BINI DICTIONARY"):
            continue

        # 2) Capture the headword = first non-space token at start of block
        m = re.match(r"^\s*(?P<head>\S+)\s*(?P<rest>.*)$", nb)
        if not m:
            continue
        head = m.group("head").strip()
        rest = m.group("rest").strip()

        # 3) If a pronunciation bracket immediately follows the headword, drop it
        rest = re.sub(r"^\[[^\]]+\]\s*", "", rest)

        # 4) Tidy leading separators (| : ; , . dashes) before the gloss
        rest = re.sub(r"^[\s\|\:\;\,\.\-–—]+", "", rest)

        rows.append((head, rest))
    return rows

def main():
    with open(INPUT_TXT, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()

    rows = extract_pairs(text)

    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as out:
        w = csv.writer(out)
        w.writerow(["bini", "english"])
        w.writerows(rows)

    print(f"Done. Wrote {len(rows)} rows to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


In [ ]:
text2 = pytesseract.image_to_string("/home/kinged/codes/dala/group_a/bini_pdf_as_img/test.png0001-015.png")
print(text2)

been given tone-marks of their own in noun-headings, but not in
grammatical elements like (e)t-, (e)v-, etc., nor in sentences.

The sign > has occasionally been used where a word had to be broken
up at the end of a line, for purely technical reasons.

ORDER AND Form OF ITEMS

The letters follow one another in the following order: a, b, d, e, ¢, f,
g, gb, x, h, i,k, kp, 1, m, n, nw, ny, 0, 9, p, r, rh, f, f, s, t, u, v, 6, w, x, y, z.

In the case of words differing in tone only, the items with high tones
are placed first, then those containing both high and low (or mid) tones,
then the words with low tones only, and last those with rises or falls.

Unnasalised vowels have no precedence over nasalised ones, except
where the tones are the same.

Nouns beginning with e-, i.e. an e- prefix which occurs mostly in a

context and is often not pronounced when isolated, are found under the
vowel e, but grammatical elements like (e)n-, (e)t-, etc. are entered
under the appropriate consonant.
En

^\n\w+ - first word that comes after a newlin
